In [3]:
import pandas as pd


def generar_csv_telco_analizable(
    ruta_csv_entrada: str,
    ruta_csv_salida: str,
    nombre_col_label: str = "label"
) -> None:
    """
    Genera un CSV tabular analizable a partir de Telco Customer Churn (IBM).

    Características del CSV final:
    - Todas las features numéricas (incluye one-hot encoding)
    - Sin NaN
    - Columna 'label' textual ('churn' / 'no-churn') COMO ÚLTIMA COLUMNA
    - Listo para cargar en el pipeline (split, escalado, SMOTE/PC-SMOTE después)

    No realiza train/test split.
    """

    # --------------------------------------------------
    # 1) Carga
    # --------------------------------------------------
    df = pd.read_csv(ruta_csv_entrada)

    # --------------------------------------------------
    # 2) Eliminar identificador
    # --------------------------------------------------
    if "customerID" in df.columns:
        df = df.drop(columns=["customerID"])

    # --------------------------------------------------
    # 3) Limpiar TotalCharges
    # --------------------------------------------------
    if "TotalCharges" in df.columns:
        df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
        mediana_total = df["TotalCharges"].median()
        df["TotalCharges"] = df["TotalCharges"].fillna(mediana_total)

    # --------------------------------------------------
    # 4) Crear label textual
    # --------------------------------------------------
    if "Churn" not in df.columns:
        raise ValueError("No se encontró la columna 'Churn' en el CSV de entrada.")

    mapa_label = {"Yes": "churn", "No": "no-churn"}
    df[nombre_col_label] = df["Churn"].map(mapa_label)

    if df[nombre_col_label].isna().any():
        valores = df["Churn"].dropna().unique()
        raise ValueError(f"Valores inesperados en 'Churn': {valores}")

    df = df.drop(columns=["Churn"])

    # --------------------------------------------------
    # 5) Variables binarias a 0/1
    # --------------------------------------------------

    # SeniorCitizen (ya es 0/1, se fuerza a int)
    if "SeniorCitizen" in df.columns:
        df["SeniorCitizen"] = (
            pd.to_numeric(df["SeniorCitizen"], errors="coerce")
            .fillna(0)
            .astype(int)
        )

    # gender
    if "gender" in df.columns:
        df["gender"] = df["gender"].map({"Female": 0, "Male": 1})
        if df["gender"].isna().any():
            vals = df["gender"].dropna().unique()
            raise ValueError(f"Valores inesperados en 'gender': {vals}")

    # Binarias Yes / No
    binarias_yes_no = ["Partner", "Dependents", "PhoneService", "PaperlessBilling"]
    for col in binarias_yes_no:
        if col in df.columns:
            df[col] = df[col].map({"No": 0, "Yes": 1})
            if df[col].isna().any():
                vals = df[col].dropna().unique()
                raise ValueError(f"Valores inesperados en '{col}': {vals}")

    # --------------------------------------------------
    # 6) One-hot encoding de categóricas nominales
    # --------------------------------------------------
    categoricas = [
        "MultipleLines", "InternetService", "OnlineSecurity",
        "OnlineBackup", "DeviceProtection", "TechSupport",
        "StreamingTV", "StreamingMovies", "Contract", "PaymentMethod"
    ]

    categoricas_presentes = [c for c in categoricas if c in df.columns]

    df = pd.get_dummies(
        df,
        columns=categoricas_presentes,
        drop_first=True
    )

    # --------------------------------------------------
    # 7) Reordenar columnas → label al final
    # --------------------------------------------------
    if nombre_col_label not in df.columns:
        raise ValueError("No se generó la columna label.")

    columnas_features = [c for c in df.columns if c != nombre_col_label]
    columnas_finales = columnas_features + [nombre_col_label]
    df = df[columnas_finales]

    # --------------------------------------------------
    # 8) Validaciones finales
    # --------------------------------------------------

    # Sin NaN
    if df.isna().any().any():
        cols_nan = [c for c in df.columns if df[c].isna().any()]
        raise ValueError(f"Quedaron NaN en el dataset final. Columnas: {cols_nan}")

    # Todo numérico salvo label
    for c in columnas_features:
        if not pd.api.types.is_numeric_dtype(df[c]):
            raise ValueError(f"Columna no numérica detectada: {c}")

    # --------------------------------------------------
    # 9) Guardar CSV final
    # --------------------------------------------------
    df.to_csv(ruta_csv_salida, index=False)

    print("✅ CSV analizable generado correctamente")
    print("   Ruta:", ruta_csv_salida)
    print("   Filas:", len(df))
    print("   Columnas:", len(df.columns))
    print("   Última columna:", df.columns[-1])
    print("   Distribución label:", df[nombre_col_label].value_counts().to_dict())


generar_csv_telco_analizable(
    ruta_csv_entrada="telco_churn_original.csv",
    ruta_csv_salida="telco_churn_analizable.csv",
    nombre_col_label="label"
)


✅ CSV analizable generado correctamente
   Ruta: telco_churn_analizable.csv
   Filas: 7043
   Columnas: 31
   Última columna: label
   Distribución label: {'no-churn': 5174, 'churn': 1869}
